In [23]:
# image_classification_preprocess_sub.py

import os
import numpy as np
import pandas as pd
import zipfile
import logging


def exec_process(pm):
    logging.info('[hunmin log] the start line of the function [exec_process]')

    '''
    class PM:
        def __init__(self):
            self.source_path = './'
            self.target_path = './meta_data'

        pm = PM()
        exec_process(pm)
    '''

    # 저장 파일 확인
    list_files_directories(pm.source_path)  
    
    # pm.source_path의 dataset.zip 파일을
    # pm.target_path 경로에 압축해제
    my_zip_path = os.path.join(pm.source_path,'dataset.zip')
    extract_zip_file = zipfile.ZipFile(my_zip_path)  # <zipfile.ZipFile filename='./dataset.zip' mode='r'>
    extract_zip_file.extractall(pm.target_path)
    extract_zip_file.close()
    

    # 저장 파일 확인
    list_files_directories(pm.target_path)

    logging.info('[hunmin log] the finish line of the function [exec_process]')


# 저장 파일 확인
def list_files_directories(path):
    # Get the list of all files and directories in current working directory
    dir_list = os.listdir(path)
    logging.info('[hunmin log] ★ Files and directories in {} :'.format(path))
    logging.info('[hunmin log] ★ dir_list : {}'.format(dir_list))  



In [24]:
# image_classification_preprocess.py

'''
from image_classification_preprocess_sub import exec_process
'''

import logging

def process_for_train(pm):   # 데이터셋 준비(Dataset Setup) 
    exec_process(pm)      # 저장 파일 확인하기
    logging.info('[hunmin log] the end line of the function [process_for_train]')
    

def init_svc(im, rule):     # 전처리 객체 불러오기
    return {}


def transform(df, params, batch_id):   #추론 데이터 전처리(Data Preprocessing)(생략 가능)
    logging.info('[hunmin log] df.shape : {}'.format(df.shape))
    logging.info('[hunmin log] type(df) : {}'.format(type(df)))
    logging.info('[hunmin log] the end line of the function [transform]')
    return df

In [25]:
# train.py
import logging, os
"""
from train_sub import exec_train
import t3qai_client as tc
from t3qai_client import T3QAI_TRAIN_OUTPUT_PATH, T3QAI_TRAIN_MODEL_PATH, \
                            T3QAI_TRAIN_DATA_PATH, T3QAI_TEST_DATA_PATH, T3QAI_MODULE_PATH
"""

def main():
    result = None
    result_msg = "success"
    tc.train_start()
    try:
        train()
    except Exception as e:
        result = e
        result_msg = e
        logging.info('error log : {}'.format(e))
    tc.train_finish(result, result_msg)

def train():
    exec_train()
    logging.info('[hunmin log] the end line of the function [train]')

#if __name__ == '__main__':
#    main()

In [26]:
#  train_sub.py

"""
import t3qai_client as tc
from t3qai_client import T3QAI_TRAIN_OUTPUT_PATH, T3QAI_TRAIN_MODEL_PATH, \
                            T3QAI_TRAIN_DATA_PATH, T3QAI_TEST_DATA_PATH, T3QAI_MODULE_PATH
"""
# Imports
import os
import numpy as np
import logging
import matplotlib.pyplot as plt
import tensorflow as tf
from kiwipiepy import Kiwi


# 저장 파일 확인
def list_files_directories(path):
    # Get the list of all files and directories in current working directory
    dir_list = os.listdir(path)
    logging.info('[hunmin log] Files and directories in {} :'.format(path))
    logging.info('[hunmin log] dir_list : {}'.format(dir_list))

## user algorithm 
# T3Q.ai 공통, 알고리즘 파라미터 불러오기(dictionary 형태)
# params = tc.train_load_param()
# logging.info('params : {}'.format(params))

gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus, 'GPU')
        logging.info('[hunmin log] gpu set complete')
        logging.info('[hunmin log] num of gpu: {}'.format(len(gpus)))
    
    except RuntimeError as e:
        logging.info('[hunmin log] gpu set failed')
        logging.info(e)

# 저장 파일 확인
def list_files_directories(path):
    # Get the list of all files and directories in current working directory
    dir_list = os.listdir(path)
    logging.info('[hunmin log] Files and directories in {} :'.format(path))
    logging.info('[hunmin log] dir_list : {}'.format(dir_list))


#불용처리 함수
in_person_stopwords='재배법 수확량 수확 분야 조사 지역 특성 분석 개발 방법 으로 유형 위한 규모 연구 항목 에서 구축 자료 평가 처리 이용 재료 활용 작업 구조 지표 기준 설계 조건 유용 성분 성능 개소'
logging.info('[stop words : default] : {}'.format(in_person_stopwords)) 

stop_words = in_person_stopwords
stop_words = stop_words.split(' ')

# kiwi 이용한  BERTopic
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.analyze(sent) # 형태소 출력
        forms_list = [token.form for token in word_tokens[0][0]] # '(토큰화결과,형태, , ) > [0][0] 뽑기
        #word_tokens = self.tagger.morphs(sent)
        #word_tokens = self.tagger.nouns(sent)
        #result = [word for word in word_tokens if len(word) > 1]
        result = [word for word in forms_list if len(word) > 1]
        final_result = remove_stopwords_from_list(result, stop_words)
        last_result = [re_word for re_word in final_result if len(re_word) > 1]
        return last_result
    
def remove_stopwords_from_list(content_list, stopwords):
    """Remove stopwords"""
    filtered_list = []
    for sentence in content_list:
        filtered_sentence = " ".join([word for word in sentence.split() if word not in stopwords])
        filtered_list.append(filtered_sentence)
    return filtered_list

def exec_train():
    logging.info('[hunmin log] the start line of the function [exec_train]')
    logging.info('[hunmin log] T3QAI_TRAIN_DATA_PATH : {}'.format(T3QAI_TRAIN_DATA_PATH))
    
    # 저장 파일 확인
    list_files_directories(T3QAI_TRAIN_DATA_PATH)
    my_path = os.path.join('./dataset', '') + '/'
    
    # 카테고리
    dataset=['dataset'] ##335개의 문서가 한 줄로 들어간 dataset.txt
    dataset_num = len(dataset) #1 : dataset.txt

    # 경로에 있는 문서데이터(txt에 한 줄 씩 335개 있는 1개 문서)를 load하고 dataset_numpy list에 추가한다.
    dataset_txt = []
    for i in range (dataset_num):
        ad = my_path + str(dataset[i]) + '.txt'
        dataset_txt.append(ad)
  
    for i in range (dataset_num):
        logging.info('[hunmin log] : {}'.format(dataset_txt[i]))
        
    text_file = my_path + str(dataset[i]) + '.txt'   

    documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]
    logging.info('[hunmin log] Input 산림청 첫번째 문서 확인 예시: {}'.format(documents[0]))

    preprocessed_documents = []
    logging.info('[hunmin log] 총 문서 개수: {}'.format(len(documents)))

    for line in tqdm(documents):
    # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
        if line and not line.replace(' ', '').isdecimal():
            preprocessed_documents.append(line)

    ###########################################################################
    ## 불용어 제거 단어
    ## in_person_stopwords = '재배법 수확량 수확 분야 조사 지역 특성 분석 개발 방법 으로 유형 위한 규모 연구 항목 에서 구축 자료 평가 처리 이용 재료 활용 작업 구조 지표 기준 설계 조건 유용 성분 성능 개소'
    ## stop_words = in_person_stopwords
    ## stop_words = stop_words.split(' ')
    ## logging.info('[hunmin log]  불용어 제거 단어: {}'.format(stop_words))
    logging.info('[hunmin log]  불용어 제거 단어: {}'.format(stop_words))
    ###########################################################################

    # 모델 구축
    # 단일 gpu 혹은 cpu학습
    num_classes = 1

    if len(gpus) < 2:
        model = model_build_and_compile(num_classes)
    # multi-gpu
    else:
        # strategy = tf.distribute.MirroredStrategy()
        logging.info('[humin log] gpu devices num {}'.format(strategy.num_replicas_in_sync))
        with strategy.scope():
            model = model_build_and_compile(num_classes)

    # Create topic model
    topics, probs = model.fit_transform(preprocessed_documents)

    ###########################################################################
    ## 플랫폼 시각화
    ###########################################################################

    # #모델 저장하기
    # logging.info('[hunmin log] T3QAI_BERTopic_MODEL_PATH : {}'.format(T3QAI_TRAIN_MODEL_PATH))
    # custom_model_name= 'bertopic_model' # params['model_name'])
    # logging.info('[custaom_model_name log] : {}'.format(custom_model_name))
    # model.save(os.path.join(T3QAI_TRAIN_MODEL_PATH, custom_model_name),serialization="safetensors", save_embedding_model=embedding_model)
   
    # 저장 파일 확인
    logging.info('[hunmin log] ★모델저장 완료★')
    list_files_directories(T3QAI_TRAIN_DATA_PATH)

    logging.info('[hunmin log] the finish line of the function [exec_train]')

    # 모델 토픽 결과 출력
    topics: List[int] = None
    top_n_topics: int = None

    # 전체 주제 수 출력하기
    freq_df = model.get_topic_freq()
    freq_df = freq_df.loc[freq_df.Topic != -1, :]

    if topics is not None:
        topics = list(topics)
    elif top_n_topics is not None:
        topics = sorted(freq_df.Topic.to_list()[:top_n_topics])
    else:
        topics = sorted(freq_df.Topic.to_list()[0:])

    logging.info('[★ 주제 수 ] : {} 개로 예상합니다.'.format(len(topics)))
    
    print('\n')
    print('Result topic 결과입니다.')
    model_get_topic_info(model)
    
    #model_visualize_barchart(model)
    #visualize_topics2(model)

    # Show wordcloud
    # create_wordcloud(model, topic=1)
    filepath1 = os.path.join(T3QAI_TRAIN_OUTPUT_PATH, "heatmap.html")
    filepath2 = os.path.join(T3QAI_TRAIN_OUTPUT_PATH, "visualize_barchart.html")
    
    fig = model.visualize_heatmap()
    fig.write_html(filepath1)
    
    fig = model.visualize_barchart()
    fig.write_html(filepath2)
    
    ###################################################################################################################    
    #  def visualize_documents2(docs)
    ###################################################################################################################
    from typing import List
    from typing import Union

    import pandas as pd
    import numpy as np
    import plotly.graph_objects as go

    def visualize_documents2(topic_model,
                        docs: List[str],
                        topics: List[int] = None,
                        embeddings: np.ndarray = None,
                        reduced_embeddings: np.ndarray = None,
                        sample: float = None,
                        hide_annotations: bool = False,
                        hide_document_hover: bool = False,
                        custom_labels: Union[bool, str] = False,
                        title: str = "<b>Documents and Topics</b>",
                        width: int = 1200,
                        height: int = 750):


        # topic_per_doc = model.topics_

        doc_info = model.get_document_info(docs)
        topic_per_doc = doc_info['Topic']

        # Sample the data to optimize for visualization and dimensionality reduction
        if sample is None or sample > 1:
            sample = 1

        indices = []

        for topic in set(topic_per_doc):
            s = np.where(np.array(topic_per_doc) == topic)[0]        # np.where 조건을 만족하는 값 찾기
            size = len(s) if len(s) < 100 else int(len(s) * sample)
            np.random.seed(42)
            indices.extend(np.random.choice(s, size=size, replace=False))    # 랜덤으로 선택, 결과 값 달라지기 때문에 umap에서 42고정
        indices = np.array(indices)

        print('indices :' , indices)

        #print('len(indices)', len(indices))

        df = pd.DataFrame({"topic": np.array(topic_per_doc)[indices]})
        #df["doc_psg"] = [docs[index] for index in indices]
        df["topic"] = [topic_per_doc[index] for index in indices]
        df["doc"] = [f"문서번호 : {i}_" + "".join(docs[i][0:20]) for i in indices]


        # Extract embeddings if not already done
        if sample is None:
            if embeddings is None and reduced_embeddings is None:
                embeddings_to_reduce = model._extract_embeddings(df.doc.to_list(), method="document")
            else:
                embeddings_to_reduce = embeddings
        else:
            if embeddings is not None:
                embeddings_to_reduce = embeddings[indices]
            elif embeddings is None and reduced_embeddings is None:
                embeddings_to_reduce = model._extract_embeddings(df.doc.to_list(), method="document")

        print('차원 감소 임베딩:', reduced_embeddings)

        # Reduce input embeddings
        if reduced_embeddings is None:
            umap_model = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine',random_state=42).fit(embeddings_to_reduce)
            embeddings_2d = umap_model.embedding_
        elif sample is not None and reduced_embeddings is not None:
            embeddings_2d = reduced_embeddings[indices]
        elif sample is None and reduced_embeddings is not None:
            embeddings_2d = reduced_embeddings

        print(embeddings_2d)

        unique_topics = set(topic_per_doc)
        if topics is None:
            topics = unique_topics

        # Combine data
        df["x"] = embeddings_2d[:, 0]
        df["y"] = embeddings_2d[:, 1]

        # Prepare text and names 주제 이름 넣어주기
        if isinstance(custom_labels, str):
            names = [[[str(topic), None]] + model.topic_aspects_[custom_labels][topic] for topic in unique_topics]
            names = ["_".join([label[0] for label in labels[:4]]) for labels in names]
            names = [label if len(label) < 30 else label[:27] + "..." for label in names]
        elif model.custom_labels_ is not None and custom_labels:
            names = [model.custom_labels_[topic + model._outliers] for topic in unique_topics]
        else:
            names = [f"{topic}_" + "_".join([word for word, value in model.get_topic(topic)][:3]) for topic in unique_topics]
    
        # Visualize
        fig = go.Figure()

        # Outliers and non-selected topics
        non_selected_topics = set(unique_topics).difference(topics)

        if len(non_selected_topics) == 0:
            non_selected_topics = [-1]

        selection = df.loc[df.topic.isin(non_selected_topics), :]
        selection["text"] = ""
        selection.loc[len(selection), :] = [None, None, selection.x.mean(), selection.y.mean(), "Other documents"]

        fig.add_trace(
            go.Scattergl(
                x=selection.x,
                y=selection.y,
                hovertext=selection.doc if not hide_document_hover else None,
                hoverinfo="text",
                mode='markers+text',
                name="other",
                showlegend=False,
                marker=dict(color='#CFD8DC', size=5, opacity=0.5)
            )
        )

        # Selected topics
        for name, topic in zip(names, unique_topics): # 0_산림_목재_참나무, 0 
            print(topic)
            if topic in topics and topic != -1:
                selection = df.loc[df.topic == topic, :]
                print(selection)
                selection["text"] = ""

                if not hide_annotations:
                    selection.loc[len(selection), :] = [None, None, selection.x.mean(), selection.y.mean(), name]

                fig.add_trace(
                    go.Scattergl(
                        x=selection.x,
                        y=selection.y,
                        hovertext=selection.doc if not hide_document_hover else None,
                        hoverinfo="text",
                        text=selection.text,
                        mode='markers+text',
                        name=name,
                        textfont=dict(
                            size=12,
                        ),
                        marker=dict(size=5, opacity=0.5)
                    )
                )

        # Add grid in a 'plus' shape
        x_range = (df.x.min() - abs((df.x.min()) * .15), df.x.max() + abs((df.x.max()) * .15))
        y_range = (df.y.min() - abs((df.y.min()) * .15), df.y.max() + abs((df.y.max()) * .15))
        fig.add_shape(type="line",
                    x0=sum(x_range) / 2, y0=y_range[0], x1=sum(x_range) / 2, y1=y_range[1],
                    line=dict(color="#CFD8DC", width=2))
        fig.add_shape(type="line",
                    x0=x_range[0], y0=sum(y_range) / 2, x1=x_range[1], y1=sum(y_range) / 2,
                    line=dict(color="#9E9E9E", width=2))
        fig.add_annotation(x=x_range[0], y=sum(y_range) / 2, text="D1", showarrow=False, yshift=10)
        fig.add_annotation(y=y_range[1], x=sum(x_range) / 2, text="D2", showarrow=False, xshift=10)

        # Stylize layout
        fig.update_layout(
            template="simple_white",
            title={
                'text': f"{title}",
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top',
                'font': dict(
                    size=22,
                    color="Black")
            },
            width=width,
            height=height
        )

        fig.update_xaxes(visible=False)
        fig.update_yaxes(visible=False)

        return fig
    
    filepath3 = os.path.join(T3QAI_TRAIN_OUTPUT_PATH, "visualize_document.html")
    fig = visualize_documents2(model, preprocessed_documents)
    fig.write_html(filepath3)

###########################################################################
## exec_train() 호출 함수 끝
###########################################################################

from konlpy.tag import Mecab
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

#custom_tokenizer = CustomTokenizer(Mecab(dicpath=r'C:\mecab\mecab-ko-dic'))
custom_tokenizer = CustomTokenizer(Kiwi())

vectorizer = CountVectorizer( tokenizer=custom_tokenizer, max_features=3000,  ngram_range=(1, 1))

from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

# ## user algorithm 
# # T3Q.ai 공통, 알고리즘 파라미터 불러오기(dictionary 형태)
# params = tc.train_load_param()
# logging.info('params : {}'.format(params))

n_neighbors = 10
n_components = 5
min_dist = 0.2
random_seed = 42
min_cluster_size = 2


# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=n_neighbors, n_components=n_components, min_dist=min_dist, metric='cosine',random_state=random_seed)

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = vectorizer
#CountVectorizer(tokenizer=custom_tokenizer, max_features=3000,  ngram_range=(1, 1))

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# Step 6 - (Optional) Fine-tune topic representations with 
# a `bertopic.representation` model
representation_model = KeyBERTInspired()

def model_build_and_compile(num_classes):
    #모델 구축
    # All steps together
    
    # 사용자로부터 입력 받기
    user_input = 'auto'
    nr_topics_input = 'auto'
    # # 입력 값이 숫자인지 문자열인지 확인
    # if user_input.isdigit():
    #     # 입력 값이 숫자일 경우 실행할 코드
    #     nr_topics_input = int(params.get('n_topics', '10')) # params['n_topics'])
    # else:
    #     # 입력 값이 문자열일 경우 실행할 코드
    #     nr_topics_input = str(params.get('n_topics', 'auto')) # params['n_topics'])
    
    
    logging.info('[n_topics] : {} '.format(nr_topics_input))
    # top_n_words_input = int(params.get('top_n_words', '10')) # params['top_n_words'])
    top_n_words_input = 10
    logging.info('[top_n_words] : {} '.format(top_n_words_input))
    
    model = BERTopic(
    nr_topics=nr_topics_input,
    top_n_words=top_n_words_input,
    embedding_model=embedding_model,          # Step 1 - Extract embeddings
    umap_model=umap_model,                    # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,              # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
    representation_model=representation_model, # Step 6 - (Optional) Fine-tune topic represenations
    calculate_probabilities=True              # 문서당 할당된 주제의 확률 대신 문서당 모든 주제의 확률을 계산합니다. 문서가 많은 경우(> 100_000) 주제 추출 속도가 느려질 수 있습니다. 
                                                # 참고: false인 경우 해당 시각화 방법을 사용할 수 없습니다 visualize_probabilities. 
                                                # 참고: 이는 HDBSCAN에 사용된 주제 확률의 근사치이며 정확한 표현은 아닙니다.
    )

    return model
    
#######################
## 시각화 구현
#######################
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 토픽 정보 출력 함수
def model_get_topic_info(model):
    logging.info('[★ Result topic 결과 준비가 완료되었습니다.]')
    print(model.get_topic_info())
    print(model.get_topic_info()['Count'])
    print(model.get_topic_info()['Name'])
    print(model.get_topic_info()['Representation'])
    print(model.get_topic_info()['Representative_Docs'])
    

# BERTopic barchart 결과화면
def model_visualize_barchart(model):
    model.visualize_barchart(top_n_topics=50, n_words=5, custom_labels=model.set_topic_labels)

#워드 클라우드 시각화 함수
def create_wordcloud(model, topic):
    text = {word: value for word, value in model.get_topic(topic)}
    logging.info('[★ Word Cloud 시각화 결과 화면 준비가 완료되었습니다.]')
    print(text)
    FONT_PATH =r"malgun.ttf"
    
    wc = WordCloud(font_path=FONT_PATH,background_color="white", max_words=1000)
    wc.generate_from_frequencies(text)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
 
    filepath = os.path.join(T3QAI_TRAIN_OUTPUT_PATH, 'wordcloud.jpg')
    plt.savefig(filepath)


INFO:root:[stop words : default] : 재배법 수확량 수확 분야 조사 지역 특성 분석 개발 방법 으로 유형 위한 규모 연구 항목 에서 구축 자료 평가 처리 이용 재료 활용 작업 구조 지표 기준 설계 조건 유용 성분 성능 개소
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


In [27]:
# inference_service.py

"""
from inference_service_sub import exec_init_model, exec_inference_dataframe, exec_inference_file
"""

import logging
logger = logging.getLogger()
logger.setLevel('INFO')

def init_model():
    params = exec_init_model()
    logging.info('[hunmin log] the end line of the function [init_model]')
    return { **params }


def inference_dataframe(df, model_info_dict):
    result = exec_inference_dataframe(df, model_info_dict)
    logging.info('[hunmin log] the end line of the function [inference_dataframe]')
    return { **result }


def inference_file(files, model_info_dict):
    result = exec_inference_file(files, model_info_dict)
    logging.info('[hunmin log] the end line of the function [inference_file]')
    return result

In [28]:
# inference_service_sub.py
import os
import io
import base64
import logging
import numpy as np
from tensorflow.keras.models import load_model
from bertopic import BERTopic
from PIL import Image

"""
from t3qai_client import DownloadFile
import t3qai_client as tc
from t3qai_client import T3QAI_TRAIN_OUTPUT_PATH, T3QAI_TRAIN_MODEL_PATH, T3QAI_TRAIN_DATA_PATH, \
                            T3QAI_TEST_DATA_PATH, T3QAI_MODULE_PATH, T3QAI_INIT_MODEL_PATH
"""


def exec_init_model():
    model_path = os.path.join(T3QAI_INIT_MODEL_PATH, 'bertopic_model')
    model = BERTopic.load(model_path)
    model_info_dict = {
        "model": model
    }
    return model_info_dict

def exec_inference_dataframe(df, model_info_dict):
    
    logging.info('[hunmin log] the start line of the function [exec_inference_dataframe]')
    
    ## 학습 모델 준비
    model = model_info_dict['model'] 

    # input 데이터 값 (전처리 된 것이 들어감)
    new_doc = df.iloc[0,0]
    # >>> 'model': <bertopic._bertopic.BERTopic object at 0x000001B7E93F8B48>}
    
    # 문서 preprocess 단계 >>> v0.2단계 생략 - 결과만 우선 테스트 중
    print(new_doc)

    # Topic transform & predict
    topics, probs = model.transform([new_doc])
    #print('예측한 토픽 번호 :', topics)

    result = {'inference' : topics}
    logging.info('[hunmin log] 주제 예측 결과 >> result : {}'.format(result))

    return result

def exec_inference_file(files, model_info_dict):
    
    """파일기반 추론함수는 files와 로드한 model을 전달받습니다."""
    logging.info('[hunmin log] the start line of the function [exec_inference_file]')
    model = model_info_dict['model']
 
    inference_result = []
    
    for one_file in files:
        logging.info(f'[hunmin log] inference: {one_file.filename}')
        inference_file = one_file.file
        new_doc = inference_file

        logging.info(f'[hunmin log] predict: {one_file.filename}')

        # data predict
        # Topic transform & predict
        topics, probs = model.transform([new_doc])
        #print('예측한 토픽 번호 :', topics)
        
        inference_result.append(topics)
        
#     result = [DownloadFile(file_path=T3QAI_TRAIN_OUTPUT_PATH + '/Accuracy_Loss.png', file_name='result.jpg'), 
#               DownloadFile(file_path=T3QAI_TRAIN_OUTPUT_PATH + '/Accuracy_Loss.png', file_name='result2.jpg')]

    result = {'inference' : inference_result}
    return result

In [29]:
import os
import shutil
import tempfile
import base64
import pandas as pd

import ipywidgets
from ipywidgets import FileUpload
from IPython.display import FileLink

# t3qai_client 클래스: t3qai_client 객체
class t3qai_client:
    def train_start(self):
        return None

    def train_finish(self, result, result_msg):
        if result_msg != "success":
            raise Exception(result_msg)
        else:
            logging.info('Error 확인 : {}'.format(result))
            logging.info("train finish")

    def train_load_param(self):
        '''set_param'''
        epoch = 5
        batch_size = 16
        params = {"epoch" : epoch, 'batch_size' : batch_size}
        return { **params }
    


class PM:
    def __init__(self):
        self.source_path = './'
        self.target_path = './meta_data'
        
class UploadFile:
    def __init__(self, file, filename):
        self.file = file
        self.filename = filename

def DownloadFile(file_name, file_obj = None, file_path = None):
    file_route = './meta_data/DownloadFiles'
    os.makedirs(file_route, exist_ok = True)
    file_dir = os.path.join(file_route, file_name)
    if (file_obj == None) == (file_path == None):
        Err_msg = "[DownloadFile Error]: Only one of the 'file_path' or 'file_obj' arguments is required."
        Err_msg += f"{0 if file_obj==None else 2} arguments entered."
        raise Exception(Err_msg)
    elif(file_obj != None):
        file_obj.seek(0)
        file_read = base64.b64encode(file_obj.read()).decode('utf-8')
        binary_file = base64.b64decode(file_read)
        with open(file_dir, 'wb') as f:
            f.write(binary_file)
    elif(file_path != None):
        shutil.copyfile(file_path, file_dir)
        
    return FileLink(file_dir)

pm = PM()

T3QAI_TRAIN_OUTPUT_PATH = './meta_data'
T3QAI_TRAIN_MODEL_PATH = './meta_data'
T3QAI_TRAIN_DATA_PATH = './meta_data'
T3QAI_TEST_DATA_PATH = './meta_data'
T3QAI_MODULE_PATH = './meta_data'
T3QAI_INIT_MODEL_PATH = './meta_data'


# t3qai_client 객체
tc = t3qai_client()
print('T3QAI_TRAIN_OUTPUT_PATH:', T3QAI_TRAIN_OUTPUT_PATH)
print('T3QAI_TRAIN_MODEL_PATH:', T3QAI_TRAIN_MODEL_PATH)
print('T3QAI_TRAIN_DATA_PATH:', T3QAI_TRAIN_DATA_PATH)
print('T3QAI_TEST_DATA_PATH:', T3QAI_TEST_DATA_PATH)
print('T3QAI_MODULE_PATH:', T3QAI_MODULE_PATH)
print('T3QAI_INIT_MODEL_PATH:', T3QAI_INIT_MODEL_PATH)

# init_svc(im, rule) 함수 입력
im = None
rule = None
# transform(df, params, batch_id) 함수 입력
batch_id = 0


import io
import pandas as pd

# 데이터 프레임 넣기 - 문서 첫 번째 라인.txt
data = [['차량용 목조교량 현장 모니터링 연구 국립미천골휴양림교 저탄소사회 구축을 목재이용도 증진 재질 정보 목구조시스템 구축 목조교량 모니터링 집성재 응력적층상판 도로교설계기준 Wood Bridge Monitoring Glulam Stress laminated deck Design code of highway bridge 주요 국산 침엽수재의 고부가가치 이용도 증진 저탄소 사회 구축을 위하여 국내 최초로 설계시공된 차량용 목조교량국립미천골휴양림교의 현장성능 모니터링을 통해 교량의 안전성 확보 장기 내구성 분석을 자료 구축 주변 환경 변화에 따른 교량 형상반응 계측 가교량 형상반응 원인 분석을 위해 트러스상현재 아래및 상판가로보 옆개소에서 온습도 변화를 계측함 산간지방에 위치하여 일교차 이에 따른 상대습도의 변화가 컸으며 계곡에 위치한 지리적 특성으로 인해 습도가 대체적으로 높게 나타남 교량 주변의 상대습도는 월월간 월월간 로 변화함에 따라 교량 부재의 성능과 형태의 변화가 있을 수 있음 나교량 받침의 상하 플레이트 간 상대변위 계측을 통해 교축방향 수평변형을 측정함 측정된 수평변형은 mm 내외로 매우 작았으며 교량 받침 신축 이음 부대설비의 안정적인 기능발현을 확인함 다트러스 하현재 가로보개소를 기준으로 하여 지상 LiDar 촬영 사진촬영 수준측량의 방법을 통해 교량의 처짐량인 수직변형을 계측함 수준측량법에 의해 유효한 결과를 도출할 수 있었으며 부재 접합부 손상에 의한 교량의 이상 처짐이 나타나지 않아 전반적으로 양호한 상태를 유지하고 있음을 확인함 라트러스 가로보의 중앙부 처짐량이 년보다 늘어났으나 안정화단계에 이른 것으로 판단되며 한계처짐량보다 매우 작아 구조적으로 안전함을 확인 시간 경과에 따른 트러스 부재 접합부 상태변화 모니터링 가부재 표면에서 발생할 수 있는 할렬 접착층 갈라짐 결함을 육안적으로 확인하였으며 현재까지 전반적으로 양호한 상태를 유지함 나개 트러스와 상하현재 접합부각개소에 대해 부재 사이의 접합부 간격 변화를 계측크랙측정기 적용하고 접합부 다우얼 주위의 갈라짐 발생을 조사함 크랙측정기의 부착성 문제가 있어 보완부재와 볼트연결완료하였으며 현재까지 전반적으로 양호한 상태를 유지함 다트러스 사재의 간격이 넓어지는 현상이 발견되어 구조안전성 검토와 간격의 경시적 분석을 진행하고 있음 시간 경과에 따른 응력적층상판 압체력 변화 모니터링 가응력적층상판의 압체부 개소에 로드셀을 설치하여 압체력 변화를 상시 모니터링 중에 있으며 현재까지 재압체 시점에 도달하지 않음 나환경 변화 반복적인 차량하중 재하에 의해 응력적층상판의 거동변화 해석을 위해 유한요소해석을 수행하였으며 압체력 변화 모니터링 결과와 연계하여 분석 수행 예정임 다트러스 부재에 부후가 진행되는 것으로 의심되는 치마버섯 자실체가 발생하여 정밀진단 중이며 진행되는 부후로 판단되면 방부처리 조치 계획임 라자외선 열화와 빗물피해 예방을 위해 차 도장 배수 공사 실시함 라 기술적 측면 가국산재를 이용하여 생산된 구조부재의 안정적인 구조성능 내구성 확보를 통하여 국산재의 새로운 수요 창출 고부가가치 이용 확대에 기여 나차량용 목조교량 설계기술 검증을 통한 기술력을 확보하고 나아가 국내 도로교설계기준국토해양부 재정에 목조교량의 반영 추진 가TRM 중점분야인 저탄소사회 구축을 목재이용도 증진 중 핵심기술분야로서 재질 정보 목구조시스템 구축 연구를 수행 나연구영역 목구조시스템 연구의 요소기술인 하이브리드 목구조시스템 연구와 관련하여 차량용 목조교량 구조시스템의 구조안정성 내구성 향상에 기여 주변 환경 변화 차량통행량에 따른 교량 형상반응 계측 가공시재료 국립미천골휴양림교교장 m 교폭 m의 차선 등교 나조사항목 교대 받침 수평변형 계측트러스 가로보 수직변형 계측및 방법처리방법 다시험규모 수평변형 개소 + 수직변형 개소시기 라분석항목 계절적인 환경 변화 차량통행량에 따른 목조교량의 수직수평변형 분석 시간 경과에 따른 트러스 부재 접합부 상태변화 모니터링 가공시재료 목조교량 아치 트러스길이 m 높이 m 개 나조사항목 트러스 부재리기다소나무 집성재할렬 표면결함 모니터링트러스 접합부 형상변화 모니터링 다시험규모 트러스부재+접합부시기분기 회 라분석항목 시간 경과에 따른 부재 표면 결함 발생 진행 분석시간 경과에 따른 접합부 간격 형상변화 분석 시간 경과에 따른 응력적층상판 압체력 변화 모니터링 가공시재료 국립미천골휴양림교교장 m 교폭 m의 차선 등교 나조사항목 응력적층상판 압체력 부후 다시험규모 압체부시기분기 회 라분석항목 응력적층상판의 압체력 변화 부후징후 점검']]
df = pd.DataFrame(data)
print('df: ', df)
print('df.dtypes:', df.dtypes)

# inference_file 함수 추론
files = []  # 업데이트 되는 파일 주제 저장

# accept 매개변수는 '*' 모두 허용, multiple 여러 파일 동시허용, description은 버튼 바로 옆에 표시
uploader = FileUpload(accept='*', multiple=True, description='select data', button_style='danger')

def uploader_change(change):
    uploader.button_style='success'          # danger에서 select data로 변경
    count = len(uploader.value)              # 업로드 된 파일 개수
    uploader._counter = count
    files.clear()                           # files.clear(): files 리스트를 비우기

    for file_num in range(count):         
        temp_data = tempfile.TemporaryFile()
        if ipywidgets.__version__[0] == '7':
            temp_data.write(list(uploader.value.values())[file_num]['content'])
            file = UploadFile(temp_data, pd.DataFrame(list(uploader.value.values())[file_num]).iloc[1,0])
        elif int(ipywidgets.__version__[0]) > 7:
            temp_data.write(uploader.value[file_num].content)
            file = UploadFile(temp_data, uploader.value[file_num].name)
        files.append(file)

uploader.observe(uploader_change, 'value')

T3QAI_TRAIN_OUTPUT_PATH: ./meta_data
T3QAI_TRAIN_MODEL_PATH: ./meta_data
T3QAI_TRAIN_DATA_PATH: ./meta_data
T3QAI_TEST_DATA_PATH: ./meta_data
T3QAI_MODULE_PATH: ./meta_data
T3QAI_INIT_MODEL_PATH: ./meta_data
df:                                                     0
0  차량용 목조교량 현장 모니터링 연구 국립미천골휴양림교 저탄소사회 구축을 목재이용도 ...
df.dtypes: 0    object
dtype: object


In [30]:
%%time
process_for_train(pm)

INFO:root:[hunmin log] the start line of the function [exec_process]
INFO:root:[hunmin log] Files and directories in ./ :
INFO:root:[hunmin log] dir_list : ['0_local_BERTopic.ipynb', '0_local_BERTopic_requirement.txt', '1_local_platform_BERTopic_test.ipynb', '2_1_1_platform_BERTopic_preprocess.py', '2_1_2_platform_BERTopic_preprocess_sub.py', '2_2_1_platform_BERTopic_train.py', '2_2_2_platform_BERTopic_train_sub.py', '2_2_3_platform_BERTopic_inference_service.py', '2_2_4_platform_BERTopic_inference_service_sub.py', 'dataset', 'dataset 원본.zip', 'dataset.zip', 'LICENSE.txt', 'meta_data', 'platform_test.ipynb', 'README.txt', 'T3Q.ai_platform_image_BERTopic', 'test_dataset', 'test_dataset -원본.zip', 'test_dataset.zip']
INFO:root:[hunmin log] Files and directories in ./meta_data :
INFO:root:[hunmin log] dir_list : ['bertopic_model', 'dataset', 'DownloadFiles', 'heatmap.html', 'test_dataset', 'visualize_barchart.html']
INFO:root:[hunmin log] the finish line of the function [exec_process]
INFO

Wall time: 26.2 ms


In [31]:
%%time
# main() 함수에서 train() 함수 실행
main()
# 에러 발생 없을 경우 None, train finish

INFO:root:[hunmin log] the start line of the function [exec_train]
INFO:root:[hunmin log] T3QAI_TRAIN_DATA_PATH : ./meta_data
INFO:root:[hunmin log] Files and directories in ./meta_data :
INFO:root:[hunmin log] dir_list : ['bertopic_model', 'dataset', 'DownloadFiles', 'heatmap.html', 'test_dataset', 'visualize_barchart.html']
INFO:root:[hunmin log] : ./dataset\/dataset.txt
INFO:root:[hunmin log] Input 산림청 첫번째 문서 확인 예시: 저비용 부산물 생산체계의 공정별 요소기술개발 가치있는 미래 산림자원 육성 효율적인 수확시스템 구축 목재수확시스템 반송기 벌채부산물 생산성 임업기계 timber harvesting system Carriage logging residue productivity forestry machines 부산물 생산작업 생력화에 필요한 효과적인 목재수확계획 기법과 공정별 요소기술개발 해당사항 없음 라 기술적 측면 가 효과적인 목질원료 생산체계 구축으로 생산성 향상 실현 나 용도별 목질원료 공급으로 원료확보 경쟁 완화 시장 안정화 다 목질원료 생산관련 임업기계장비의 활성화로 관련 업체의 산업화 가 집재장비의 성능개선으로 목재생산성 향상 비용절감 실현 나 벌채부산물의 효율적인 생산공급 체계 구축 저비용 부산물 생산체계의 공정별 요소기술 가 집재작업 생산성 향상을 반송기 공시재료조사대상 반송기 가 문헌 고찰 현장방문을 통한 집재기 반송기의 유형 분류 나 집재작업 생산성 향상을 위해 개발이 요구되는 반송기 개발계획 모델 구축 시험규모조사규모 반송기 식 가 모델 Simulation 역학분석을 통한 반송기의 적용 범위 제원 나 반송기 시제품 



Result topic 결과입니다.
    Topic  Count                            Name  \
0      -1     26              -1_낙엽송_산림_갈매나무과_나무   
1       0    212                 0_산림_낙엽송_목재_소나무   
2       1     11                  1_목재_소나무_건축_목조   
3       2     11      2_나노셀룰로오스_나노셀룰_나노셀룰로오스의_생체   
4       3      6                3_재배지_재배_침엽수림_뿌리   
5       4      6                4_생합성_광합성_바이오_생체   
6       5      6               5_oil_활성라디칼_정유_향료   
7       6      6            6_인도네시아_바이오매스_습지_바이오   
8       7      5           7_통합_성과_evaluation_구성   
9       8      5                  8_유통_수확기_농도_생산   
10      9      4                   9_통합_성과_파급_협력   
11     10      4             10_건조제_drying_건조_수분   
12     11      4         11_synthase_바이오_생합성_유전체   
13     12      4                  12_한국_미백_배지_경도   
14     13      4                 13_수출국_무역_경제_수출   
15     14      4         14_insulation_목재_주택_낙엽송   
16     15      4              15_당화액_연료_수처리_탄화수소   
17     16      4        16_일본잎갈나무_느티나무_리기다

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

차원 감소 임베딩: None
[[ 8.38991   11.79931  ]
 [12.746222  10.429168 ]
 [11.412541  10.144917 ]
 [ 5.740336  10.037787 ]
 [ 7.865338   7.7229676]
 [ 9.232457   8.936847 ]
 [11.086871  12.307781 ]
 [ 9.261339   7.1841598]
 [11.764821  11.032231 ]
 [10.815488  12.346365 ]
 [ 7.962297  10.252466 ]
 [ 8.07106   10.379875 ]
 [10.522193  12.703469 ]
 [10.883287  11.295875 ]
 [11.010291  11.523878 ]
 [ 4.1898346  9.132327 ]
 [ 5.9232     9.81907  ]
 [11.46685   11.060003 ]
 [ 9.037386   9.613567 ]
 [ 8.007433  10.311011 ]
 [ 7.046211  12.211695 ]
 [ 5.885493  10.714647 ]
 [ 4.170816   9.092423 ]
 [ 8.025678  10.358238 ]
 [10.485144  11.402495 ]
 [ 6.3985496 10.782108 ]
 [10.8737545 11.3749075]
 [ 8.565239  11.8335   ]
 [ 7.1472044 11.450905 ]
 [ 6.0467014 10.946925 ]
 [10.96516   12.031886 ]
 [12.442798  11.302525 ]
 [11.041279  12.107218 ]
 [10.368113  10.121268 ]
 [ 8.050321  10.349491 ]
 [10.478713  11.123086 ]
 [ 9.088716   7.177307 ]
 [11.97648   11.0949   ]
 [11.322404  10.085452 ]
 [ 9.2001

INFO:root:[hunmin log] the end line of the function [train]
INFO:root:Error 확인 : None
INFO:root:train finish


Wall time: 3min 50s


In [32]:
%%time
# preprocess_sub의 transform # 데이텨 변환시 메모리에 standby 시켜놓을 데이터 반환
params = init_svc(im, rule)
print(params) # BERTopic 없다



{}
Wall time: 0 ns


In [33]:
%%time
# preprocess_sub의 transform # 추론 이전에 데이터 변환
df = transform(df, params, batch_id)

INFO:root:[hunmin log] df.shape : (1, 1)
INFO:root:[hunmin log] type(df) : <class 'pandas.core.frame.DataFrame'>
INFO:root:[hunmin log] the end line of the function [transform]


Wall time: 2.1 ms


In [34]:
%%time
# inference_service의 params = exec_init_model() > dict = { 'model' : model }
model_info_dict = init_model()
print(model_info_dict) # { 'model' : model }

INFO:root:[hunmin log] the end line of the function [init_model]


{'model': <bertopic._bertopic.BERTopic object at 0x000002864E2D78C8>}
Wall time: 2.56 s


### CASE [추론 입력 타입 - 추론 출력 타입] : 총 4가지
추론 입력 타입 : DataFrame &rarr; 추론 출력 타입 : Dictionary (1가지)    
추론 입력 타입 : File &rarr; 추론 출력 타입 : Dictionary, DownloadFile, DownloadFile의 List (3가지)  

### CASE  [DataFrame - Dictionary]
DataFrame 입력에 대한 추론 결과를 딕셔너리(Dictionary) 형태로 리턴(return)

In [35]:
%%time
inference_dataframe(df, model_info_dict)

INFO:root:[hunmin log] the start line of the function [exec_inference_dataframe]


차량용 목조교량 현장 모니터링 연구 국립미천골휴양림교 저탄소사회 구축을 목재이용도 증진 재질 정보 목구조시스템 구축 목조교량 모니터링 집성재 응력적층상판 도로교설계기준 Wood Bridge Monitoring Glulam Stress laminated deck Design code of highway bridge 주요 국산 침엽수재의 고부가가치 이용도 증진 저탄소 사회 구축을 위하여 국내 최초로 설계시공된 차량용 목조교량국립미천골휴양림교의 현장성능 모니터링을 통해 교량의 안전성 확보 장기 내구성 분석을 자료 구축 주변 환경 변화에 따른 교량 형상반응 계측 가교량 형상반응 원인 분석을 위해 트러스상현재 아래및 상판가로보 옆개소에서 온습도 변화를 계측함 산간지방에 위치하여 일교차 이에 따른 상대습도의 변화가 컸으며 계곡에 위치한 지리적 특성으로 인해 습도가 대체적으로 높게 나타남 교량 주변의 상대습도는 월월간 월월간 로 변화함에 따라 교량 부재의 성능과 형태의 변화가 있을 수 있음 나교량 받침의 상하 플레이트 간 상대변위 계측을 통해 교축방향 수평변형을 측정함 측정된 수평변형은 mm 내외로 매우 작았으며 교량 받침 신축 이음 부대설비의 안정적인 기능발현을 확인함 다트러스 하현재 가로보개소를 기준으로 하여 지상 LiDar 촬영 사진촬영 수준측량의 방법을 통해 교량의 처짐량인 수직변형을 계측함 수준측량법에 의해 유효한 결과를 도출할 수 있었으며 부재 접합부 손상에 의한 교량의 이상 처짐이 나타나지 않아 전반적으로 양호한 상태를 유지하고 있음을 확인함 라트러스 가로보의 중앙부 처짐량이 년보다 늘어났으나 안정화단계에 이른 것으로 판단되며 한계처짐량보다 매우 작아 구조적으로 안전함을 확인 시간 경과에 따른 트러스 부재 접합부 상태변화 모니터링 가부재 표면에서 발생할 수 있는 할렬 접착층 갈라짐 결함을 육안적으로 확인하였으며 현재까지 전반적으로 양호한 상태를 유지함 나개 트러스와 상하현재 접합부각개소에 대해 부재 사이의 접합부 간격 변화를 계측크랙측정기 적용하고 접합부 다우

INFO:root:[hunmin log] 주제 예측 결과 >> result : {'inference': [-1]}
INFO:root:[hunmin log] the end line of the function [inference_dataframe]


Wall time: 5.17 s


{'inference': [-1]}

### CASE  [File - Dictionary]
File 에 대한 추론 결과를 딕셔너리(Dictionary) 형태로 리턴(return)

 1. 아래 Cell을 실행하면 select data 버튼이 생성됩니다.
 2. 생성된 select data 버튼을 눌러 추론할 데이터를 선택하세요.
 3. 선택 후 **inference_file(files, model_info_dict)** 을 실행하세요.

In [36]:
# uploader widget(해당 커널 output의 버튼)에 파일을 업로드 한 뒤 infernece_file으로 추론합니다.
display(uploader)

FileUpload(value=(), accept='*', button_style='danger', description='select data', multiple=True)

In [37]:
%%time
inference_file(files, model_info_dict)

INFO:root:[hunmin log] the start line of the function [exec_inference_file]
INFO:root:[hunmin log] the end line of the function [inference_file]


Wall time: 0 ns


{'inference': []}

### CASE  [File - DownloadFile / DownloadFile의 List]
File 입력에 대한 추론 결과를 DownloadFile 형태 혹은 DownloadFile의 List 형태로 리턴(return)

In [38]:
%%time
# inference의 return을 DownloadFile 으로 할때 실행합니다.
inference_result = inference_file(files, model_info_dict)

if type(inference_result) == list:
    display(*inference_result)
else:
    display(inference_result)

INFO:root:[hunmin log] the start line of the function [exec_inference_file]
INFO:root:[hunmin log] the end line of the function [inference_file]


{'inference': []}

Wall time: 0 ns
